## 2)

> Rehacer el Problema 1) considerando ahora un flujo estratificado en la aproximación de Boussinesq.
Utilice el solver BOUSS de GHOST, nuevamente con condiciones iniciales nulas para el campo de velocidad,
con forzado mecánico aleatorio con amplitud `f0 = 0.75` centrado en los modos Fourier con $k$ entre $1$ y $4$
(i.e., `kdn = 1`, `kup = 4`)
con las mismas propiedades temporales que en el problema anterior (`rand=2`, `cort=0.5`),
y sin fluctuaciones iniciales en la temperatura y sin fuentes térmicas ($c0 = s0 = 0$).
Utilice
una frecuencia de Brunt-Väisälä $N = 8$,
viscosidad cinemática y difusividad térmica $ν = κ = 2 × 10^{−3}$ y
$192 × 192 × 48$ puntos de resolución espacial en un dominio cúbico con tamaño $2π × 2π × π/2$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from common import Output
from ghost_xarray import open_dataarray


plt.rc("figure", dpi=100, figsize=(6, 3))

t_max = 25
Δt_save_spectrum = 0.5
Δt_save_field = 1.5

output = Output(
    "problema1/output/",
    dt=Δt_save_spectrum,
    balance=["time", "<v^2>", "<omega^2>", "mechanic injection rate"],
    helicity=["time", "kinetic helicity"],
    kspectrum=["k", "Ev(k)"],
    ktransfer=["k", "Tv(k)"],
    kspecpara=["kz", "Ev(kz)", "Ev_perp(kz)", "Ev_z(kz)"],
    kspecperp=["kp", "E_v(kp)", "ev_x,y(kp,kz=0)", "ev_z(kp,kz=0)"],
)

> a) Compare las resoluciones espaciales $∆x$, $∆y$ y $∆z$ del problema con las del Problema 1).
¿Como justifica los cambios en el tamaño del dominio y en la resolución?

> b) Grafique la energía cinética, la energía potencial, y el cociente entre ambas.
¿Como son las fluctuaciones térmicas comparadas con la velocidad?

> c) Grafique el espectro de energía cinética isótropo $E(k)$,
el espectro de energía cinética perpendicular $E(k⊥)$,
y el espectro de energía cinética paralelo $E(k∥)$
promediados en el tiempo, para tiempos suficientemente tardíos.
Compare los espectros con las predicciones fenomenológicas.

> d) Utilizando los espectros de energía calcule
la longitud integral isótropa $L$,
la longitud perpendicular $L⊥$,
y la longitud paralela $L∥$
en función del tiempo.
Calcule la longitud paralela obtenida por análisis dimensional
(la "longitud de empuje"), $L_B = 2π/k_B$ (con $k_B = N/U$, donde $U$ es la velocidad típica del fluido),
y compare esta longitud con la obtenida a partir del espectro, $L∥$.
Utilizando estas longitudes estime el número de Reynolds y los números de Froude paralelo y perpendicular (puede asumir $U⊥ ∼ U$).

> e) Para algún tiempo $t > t^*$,
grafique un corte de las fluctuaciones de temperatura $π$ en el plano $x − z$.
¿Qué observa? ¿Son compatibles las estructuras con lo que esperaba?